In [1]:
import re
import string
import glob

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hemant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hemant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [4]:
positional_index = {}

In [5]:
def pre_preocessing(document):
    token_list = word_tokenize(document)
    # remove puncuation and special characters
    # removing strings containing . and -
    token_list = [token for token in token_list if '.' not in token]
    token_list = [token for token in token_list if '-' not in token]
    token_list = [re.sub('[^A-Za-z0-9]+', '', token) for token in token_list]
    # if string contains any digit then its removed 
    token_list = [token for token in token_list if not any(map(str.isdigit, token))]
    # all empty strings are removed
    token_list = [str for str in token_list if str]
    # all terms converted to lower characters
    token_list = [token.lower() for token in token_list]
    
    return token_list

In [6]:
def create_positional_postings(token_list, file_no):
    global positional_index
    for pos, term in enumerate(token_list):
        postings = {}
        term = lemmatizer.lemmatize(term)
        if term in positional_index.keys():
            postings = positional_index[term]
            if file_no in postings.keys():
                postings[file_no].append(pos)
            else:
                postings[file_no] = []
                postings[file_no].append(pos)
        else:
            postings[file_no] = []
            postings[file_no].append(pos)
            positional_index[term] = postings

In [7]:
document_path = []
for folder_path in glob.glob('/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/*'):
    document_path.append(glob.glob(folder_path+'/*'))

In [8]:
document_path = [path for long_path in document_path for path in long_path]

In [9]:
positional_index = {}
for path in document_path:
    with open(path,encoding='iso-8859-15') as f:
        print(path)
        document = f.read()
        token_list = pre_preocessing(document)
        create_positional_postings(token_list, path.split('/')[-2] + '/' + path.split('/')[-1])

/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.sys.mac.hardware/50432
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/alt.atheism/53128
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/alt.atheism/53127
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.os.ms-windows.misc/9464
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.os.ms-windows.misc/9471
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.os.ms-windows.misc/9470
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.os.ms-windows.misc/9469
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/sci.med/58049
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.windows.x/66410
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.windows.x/64830
/Users/hemant/Desktop/DOR/DOR_Assignment3/assignment3/20_newsgroups/comp.windows.x/6

In [10]:
len(document_path)

22

In [11]:
positional_index

{'newsgroups': {'comp.sys.mac.hardware/50432': [0],
  'alt.atheism/53128': [12],
  'alt.atheism/53127': [10],
  'comp.os.ms-windows.misc/9464': [9],
  'comp.os.ms-windows.misc/9471': [1],
  'comp.os.ms-windows.misc/9470': [2],
  'comp.os.ms-windows.misc/9469': [0],
  'sci.med/58049': [0],
  'comp.windows.x/66410': [11],
  'comp.windows.x/64830': [8],
  'comp.windows.x/66407': [0],
  'comp.windows.x/66400': [0],
  'comp.windows.x/66409': [10],
  'comp.graphics/37913': [9],
  'comp.graphics/37914': [8],
  'comp.sys.ibm.pc.hardware/58841': [9],
  'comp.sys.ibm.pc.hardware/58840': [0],
  'comp.sys.ibm.pc.hardware/58827': [8],
  'sci.space/59904': [8],
  'sci.space/59913': [8],
  'sci.space/59873': [6],
  'sci.space/59874': [6]},
 'path': {'comp.sys.mac.hardware/50432': [1],
  'alt.atheism/53128': [1],
  'alt.atheism/53127': [0],
  'comp.os.ms-windows.misc/9464': [1],
  'comp.os.ms-windows.misc/9471': [2],
  'comp.os.ms-windows.misc/9470': [0],
  'comp.os.ms-windows.misc/9469': [1],
  'sci.

In [32]:
def query(term, positional_index):
    term = lemmatizer.lemmatize(term.lower())
    if term in positional_index.keys():
        return list(positional_index[term])
    else:
        return None

In [33]:
query('design', positional_index)

['comp.os.ms-windows.misc/9464',
 'comp.windows.x/64830',
 'sci.space/59913',
 'sci.space/59874']

In [36]:
query('newsgroup', positional_index)

['comp.os.ms-windows.misc/9464', 'comp.windows.x/66409']

In [37]:
def f_and(term1, term2, positional_index):
    docs_term1 = query(term1, positional_index)
    docs_term2 = query(term2, positional_index)
    
    lst = []
    for doc in docs_term1:
        if doc in docs_term2:
            lst.append(doc)
            
    return lst

In [38]:
f_and('design', 'newsgroup', positional_index)

['comp.os.ms-windows.misc/9464']

In [39]:
def f_or(term1, term2, positiona_index):
    docs_term1 = query(term1, positional_index)
    docs_term2 = query(term2, positional_index)
    
    return set(docs_term1 + docs_term2)

In [40]:
f_or('design', 'newsgroup', positional_index)

{'comp.os.ms-windows.misc/9464',
 'comp.windows.x/64830',
 'comp.windows.x/66409',
 'sci.space/59874',
 'sci.space/59913'}